##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

Download and install TensorFlow 2. Import TensorFlow into your program:

Note: Upgrade `pip` to install the TensorFlow 2 package. See the [install guide](https://www.tensorflow.org/install) for details.

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
tf.compat.v1.enable_eager_execution()

# After eager execution is enabled, operations are executed as they are
# defined and Tensor objects hold concrete values, which can be accessed as
# numpy.ndarray`s through the numpy() method.
assert tf.multiply(6, 7).numpy() == 42

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()    # load data
x_train, x_test = x_train / 255.0, x_test / 255.0        # normalize data

Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [3]:
# define the model
model = tf.keras.models.Sequential([                   # Sequential: concatenate function
  tf.keras.layers.Flatten(input_shape=(28, 28)),       # flaten the vector: 28 * 28 -> 784 * 1
  tf.keras.layers.Dense(128, activation='relu'),       # dense layer: a fully connected layer with 128 nodes
  tf.keras.layers.Dropout(0.2),                        # dropout probability is 0.2
  tf.keras.layers.Dense(10)                            # dense layer: this is a output layer
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [4]:
predictions = model(x_train[:1]).numpy()
predictions


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



array([[ 0.02477953,  0.85760677,  0.08163691,  0.44025543,  0.09560558,
        -0.45472717, -0.27686763,  0.24076167, -0.40844733,  0.30551165]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [5]:
tf.nn.softmax(predictions).numpy()

array([[0.08681247, 0.19965233, 0.09189142, 0.13152884, 0.09318403,
        0.05374462, 0.06420641, 0.10774123, 0.05629036, 0.1149483 ]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [7]:
loss_fn(y_train[:1], predictions).numpy()

2.9235117

In [8]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [9]:
model.fit(x_train, y_train, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 3s 53us/sample - loss: 0.2974 - accuracy: 0.9123
Epoch 2/5
60000/60000 [==============================] - 2s 36us/sample - loss: 0.1438 - accuracy: 0.9565
Epoch 3/5
60000/60000 [==============================] - 2s 36us/sample - loss: 0.1057 - accuracy: 0.9682
Epoch 4/5
60000/60000 [==============================] - 2s 34us/sample - loss: 0.0868 - accuracy: 0.9731
Epoch 5/5
60000/60000 [==============================] - 2s 34us/sample - loss: 0.0743 - accuracy: 0.9768


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)".

In [13]:
model.evaluate(x_test,  y_test, verbose=2)

10000/10000 - 0s - loss: 0.0698 - acc: 0.9782


[0.06978555842940695, 0.9782]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [14]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [15]:
probability_model(x_test[:5])

<tf.Tensor: id=68362, shape=(5, 10), dtype=float32, numpy=
array([[3.23517980e-07, 6.43183427e-08, 1.12669208e-04, 2.17031280e-04,
        5.08654317e-12, 4.22008739e-08, 7.00883375e-14, 9.99664903e-01,
        6.31446113e-08, 4.82680980e-06],
       [1.79466014e-07, 3.48315807e-03, 9.96423781e-01, 9.05289489e-05,
        8.85788974e-14, 3.53192576e-07, 1.15069565e-06, 4.40489728e-11,
        8.01954855e-07, 4.09352196e-12],
       [4.48311584e-06, 9.96351242e-01, 1.89285856e-04, 8.57415580e-05,
        2.46554351e-04, 1.21149169e-04, 1.00130819e-04, 1.64810568e-03,
        1.25051255e-03, 2.74854574e-06],
       [9.99917626e-01, 6.38003499e-08, 1.91106046e-05, 1.20090595e-08,
        4.88727530e-08, 3.71728106e-06, 1.87120404e-05, 3.25365800e-05,
        5.73872754e-08, 8.22798847e-06],
       [5.09541962e-07, 4.10765599e-09, 2.43189970e-06, 1.52355228e-09,
        9.99016404e-01, 4.17126209e-08, 8.13554522e-08, 3.01196701e-06,
        3.96261385e-07, 9.77177522e-04]], dtype=float32)>